In [1]:
# Import some useful packages
import matplotlib.pyplot as plt
import numpy as np

# Layers for FNN
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input, concatenate, add

# Layers for CNN
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D

from tensorflow.keras import datasets
from tensorflow.keras.optimizers import SGD, Adam

# For data preprocessing
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical, plot_model

In [2]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

In [3]:
# Normalization
x_train = x_train / 255
x_test = x_test / 255

In [4]:
# Reshape 
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

In [5]:
# One-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### Functional API

In [6]:
x = Input(shape=(28, 28, 1))
print(x)

Tensor("input_1:0", shape=(None, 28, 28, 1), dtype=float32)


In [7]:
conv1 = Conv2D(64, (5,5), padding='same', activation='relu')(x)
pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
flatten1 = Flatten()(pool1)

In [8]:
conv2 = Conv2D(128, (5,5), padding='same', activation='relu')(x)
pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
flatten2 = Flatten()(pool2)

In [9]:
c = concatenate([flatten1,flatten2])


In [10]:
L1 = Dense(500, activation='sigmoid')
L2 = Dense(300, activation='sigmoid')
L3 = Dense(10, activation='softmax')

In [11]:
V1 = L1(c)
V2 = L2(V1)
y = L3(V2)
print(y)

Tensor("dense_2/Identity:0", shape=(None, 10), dtype=float32)


In [12]:
model = Model(x,y)
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 64)   1664        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 28, 28, 128)  3328        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 14, 14, 64)   0           conv2d[0][0]                     
______________________________________________________________________________________________

In [13]:
model.compile(loss='mse', optimizer=Adam(), metrics=['accuracy'])


In [14]:
model.fit(x_train, y_train, batch_size=200, epochs=5, validation_data=(x_test,y_test))


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 224s 4ms/sample - loss: 0.0134 - accuracy: 0.9102 - val_loss: 0.0034 - val_accuracy: 0.9802
Epoch 2/5
60000/60000 [==============================] - 185s 3ms/sample - loss: 0.0028 - accuracy: 0.9827 - val_loss: 0.0028 - val_accuracy: 0.9821
Epoch 3/5
60000/60000 [==============================] - 188s 3ms/sample - loss: 0.0019 - accuracy: 0.9883 - val_loss: 0.0017 - val_accuracy: 0.9888
Epoch 4/5
60000/60000 [==============================] - 200s 3ms/sample - loss: 0.0013 - accuracy: 0.9919 - val_loss: 0.0017 - val_accuracy: 0.9888
Epoch 5/5
60000/60000 [==============================] - 184s 3ms/sample - loss: 0.0010 - accuracy: 0.9940 - val_loss: 0.0018 - val_accuracy: 0.9885


In [15]:
result = model.predict(x_test)

In [16]:
loss, acc = model.evaluate(x_test, y_test)

10000/10000 [==============================] - 11s 1ms/sample - loss: 0.0018 - accuracy: 0.9885


In [19]:
print(f"loss: {loss: .3f}\naccuracy rate: {acc * 100: .2f}%")

loss:  0.002
accuracy rate:  98.85%


In [20]:
model.save_weights('mnist_functionAPI_model.h5')